(**Click the icon below to open this notebook in Colab**)

[![Open InColab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiangshiyin/machine-learning-for-actuarial-science/blob/main/2025-spring/week14/notebook/demo.ipynb)

# Model Serving

## `Pickle`
Serialize a model for later usage

### Example 1 - Save a Python object to local storage

In [ ]:
import pickle

data = {
    'name': 'John Doe',
    'age': 30,
    'city': 'New York',
    'occupation': 'Software Engineer'
}

# Save this dictionary to a file
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

print("Data saved to file data.pickle.")

In [ ]:
# Load the dictionary back from the pickle file
with open('data.pickle', 'rb') as f:
    data2 = pickle.load(f)
print("Dictionary loaded successfully!")

In [ ]:
data2

In [ ]:
data2 == data

In [ ]:
id(data2), id(data)

### Example 2 - Save a ML model

In [ ]:
import pickle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dataset loaded and split successfully!")

In [ ]:
# Train a RandomForest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Model trained successfully!")

In [ ]:
# Save the trained model to a file
with open("iris_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved successfully as 'iris_model.pkl'!")

In [ ]:
# Predict with the in-memory model
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
# Print the accuracy
print("Accuracy:", accuracy)


In [ ]:
y_pred[:20]

In [ ]:
y_test[:20]

In [ ]:
# Predict with the saved model
with open('iris_model.pkl', 'rb') as f:
    model2 = pickle.load(f)

y_pred2 = model2.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)
print("Accuracy:", accuracy2)

In [ ]:
y_pred2[:20]

In [ ]:
y_test[:20]

In [ ]:
# Single data point (sepal length, sepal width, petal length, petal width)
single_data_point = [[5.1, 3.5, 1.4, 0.2]]

# Predict class for this single data point
single_prediction = model.predict(single_data_point)

# Get class name from iris dataset
predicted_class = iris.target_names[single_prediction[0]]

print(f"Predicted Class for the Single Data Point: {predicted_class}")


In [ ]:
# Predict class for this single data point
single_prediction2 = model.predict(single_data_point)

# Get class name from iris dataset
predicted_class2 = iris.target_names[single_prediction2[0]]

print(f"Predicted Class for the Single Data Point: {predicted_class2}")

In [ ]:
single_prediction

In [ ]:
iris.target_names

## `FastAPI`

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import numpy as np

app = FastAPI()

# Load trained model
with open("iris_model.pkl", "rb") as file:
    model = pickle.load(file)


# Define the expected request format
class FeaturesInput(BaseModel):
    features: list[float]


@app.post("/predict")
def predict(data: FeaturesInput):
    prediction = model.predict([data.features])
    return {"prediction": int(prediction[0])}


Run command `uvicorn fastapi_demo:app --host 0.0.0.0 --port 8000 --reload` to expose the API endpoints.
- Or run `fastapi dev fastapi_demo.py --reload`
Run `curl` command below to test the API
```
curl -X POST "http://127.0.0.1:8000/predict" \
      -H "Content-Type: application/json" \
      -d '{"features": [5.1, 3.5, 1.4, 0.2]}'
```

## `Streamlit` example

In [ ]:
import streamlit as st
import pickle
import numpy as np

# Load model
with open("iris_model.pkl", "rb") as file:
    model = pickle.load(file)

st.title("Iris Flower Classifier")

# User input
sepal_length = st.number_input("Sepal Length")
sepal_width = st.number_input("Sepal Width")
petal_length = st.number_input("Petal Length")
petal_width = st.number_input("Petal Width")

if st.button("Predict"):
    features = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    prediction = model.predict(features)[0]
    st.write(f"Predicted Class: {prediction}")


Run `streamlit run streamlit_demo.py`

# Introduction to NLP

## Preprocessing

In [1]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiangshiyin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiangshiyin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/xiangshiyin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


data = "This is a simple example to demonstrate removing stopwords using NLTK."
stopWords = set(stopwords.words('english'))

In [3]:
len(stopWords)

198

In [4]:
stopwords.words('english')[:10]

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an']

In [5]:
tokenized_data = word_tokenize(data)

In [6]:
print(f"Original text: {data}")
print(f"Tokenized text: {"|".join(tokenized_data)}")

Original text: This is a simple example to demonstrate removing stopwords using NLTK.
Tokenized text: This|is|a|simple|example|to|demonstrate|removing|stopwords|using|NLTK|.


In [7]:
filtered_tokenized_data = [
    w
    for w in tokenized_data
    if w not in stopWords
]
print(f"After removing stopwords: {filtered_tokenized_data}")

After removing stopwords: ['This', 'simple', 'example', 'demonstrate', 'removing', 'stopwords', 'using', 'NLTK', '.']


In [8]:
print(f"Original text: {data}")
print(f"Tokenized text: {"|".join(tokenized_data)}")
print(f"After removing stopwords: {"|".join(filtered_tokenized_data)}")

Original text: This is a simple example to demonstrate removing stopwords using NLTK.
Tokenized text: This|is|a|simple|example|to|demonstrate|removing|stopwords|using|NLTK|.
After removing stopwords: This|simple|example|demonstrate|removing|stopwords|using|NLTK|.


## Feature Extraction

### Bag of Words

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Sample dataset
texts = [
    "I love this product",         # positive
    "This is amazing",             # positive
    "Very happy with the result",  # positive
    "I hate this",                 # negative
    "Worst experience ever",       # negative
    "Not satisfied at all"         # negative
]

labels = [1, 1, 1, 0, 0, 0]  # 1 = positive, 0 = negative

# 2. Convert text to bag-of-words vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# 3. Show feature names
print("Feature Names (Vocabulary):")
print(vectorizer.get_feature_names_out())


Feature Names (Vocabulary):
['all' 'amazing' 'at' 'ever' 'experience' 'happy' 'hate' 'is' 'love' 'not'
 'product' 'result' 'satisfied' 'the' 'this' 'very' 'with' 'worst']


In [10]:
X.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]])

### TF-IDF

In [18]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk

# Download NLTK movie_reviews data
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

# Prepare dataset
docs = []
labels = []

for fileid in movie_reviews.fileids():
    docs.append(movie_reviews.raw(fileid))
    labels.append(movie_reviews.categories(fileid)[0])  # 'pos' or 'neg'

# Convert labels to binary format
y = [1 if label == 'pos' else 0 for label in labels]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(docs, y, test_size=0.2, random_state=42)

# Vectorize using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/xiangshiyin/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Accuracy: 0.8275
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.81      0.82       199
           1       0.82      0.84      0.83       201

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



In [20]:
feature_names = vectorizer.get_feature_names_out()
feature_names[:20]

array(['00', '000', '0009f', '007', '00s', '03', '04', '05', '05425',
       '10', '100', '1000', '10000', '100m', '101', '102', '103', '104',
       '105', '106'], dtype=object)

In [22]:
import pandas as pd

# Choose a sample document from the test set
sample_idx = 0
sample_vector = X_test_tfidf[sample_idx]

# Convert sparse vector to dense and create DataFrame
df_features = pd.DataFrame(
    data=sample_vector.toarray()[0],
    index=feature_names,
    columns=["tfidf"]
)

# Filter non-zero features and sort
df_nonzero = df_features[df_features.tfidf > 0].sort_values(by="tfidf", ascending=False)

# Show top 15 features by TF-IDF weight
print("\nTop TF-IDF features in sample test document:")
print(df_nonzero.head(15))


Top TF-IDF features in sample test document:
                 tfidf
bates         0.401204
annie         0.253364
caan          0.169454
kathy         0.148969
sledgehammer  0.127790
realises      0.117562
french        0.114832
nerve         0.107711
cake          0.104894
misery        0.102454
masterful     0.100301
basically     0.093828
prisoner      0.092226
arm           0.088148
tense         0.086170


### Word2Vec

#### Hand-craft implementation

In [11]:
import numpy as np
import re
import random

# Sample corpus
corpus = "The quick brown fox jumps over the lazy dog"

# Preprocessing: Tokenization and vocabulary building
tokens = re.findall(r'\b\w+\b', corpus.lower())
vocab = set(tokens)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(vocab)

In [12]:
word_to_idx

{'dog': 0,
 'brown': 1,
 'fox': 2,
 'over': 3,
 'lazy': 4,
 'the': 5,
 'jumps': 6,
 'quick': 7}

In [13]:
idx_to_word

{0: 'dog',
 1: 'brown',
 2: 'fox',
 3: 'over',
 4: 'lazy',
 5: 'the',
 6: 'jumps',
 7: 'quick'}

In [14]:
tokens

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [15]:
# Generate training data
def generate_training_data(tokens, window_size):
    training_data = []
    for idx, target_word in enumerate(tokens):
        target_idx = word_to_idx[target_word]
        context_range = list(range(max(0, idx - window_size), idx)) + \
                        list(range(idx + 1, min(len(tokens), idx + window_size + 1)))
        for context_idx in context_range:
            context_word = tokens[context_idx]
            context_word_idx = word_to_idx[context_word]
            training_data.append((target_idx, context_word_idx))
    return training_data

window_size = 2
training_data = generate_training_data(tokens, window_size)


In [16]:
# Inspect the training data
print(f"Corpus: {corpus}")
print([
    (idx_to_word[t[0]], idx_to_word[t[1]])
    for t in training_data
])

Corpus: The quick brown fox jumps over the lazy dog
[('the', 'quick'), ('the', 'brown'), ('quick', 'the'), ('quick', 'brown'), ('quick', 'fox'), ('brown', 'the'), ('brown', 'quick'), ('brown', 'fox'), ('brown', 'jumps'), ('fox', 'quick'), ('fox', 'brown'), ('fox', 'jumps'), ('fox', 'over'), ('jumps', 'brown'), ('jumps', 'fox'), ('jumps', 'over'), ('jumps', 'the'), ('over', 'fox'), ('over', 'jumps'), ('over', 'the'), ('over', 'lazy'), ('the', 'jumps'), ('the', 'over'), ('the', 'lazy'), ('the', 'dog'), ('lazy', 'over'), ('lazy', 'the'), ('lazy', 'dog'), ('dog', 'the'), ('dog', 'lazy')]


In [17]:
# Initialize parameters
embedding_dim = 10
W1 = np.random.randn(vocab_size, embedding_dim)
W2 = np.random.randn(embedding_dim, vocab_size)

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Training parameters
epochs = 1000
learning_rate = 0.01
num_negative_samples = 2

# Training loop
for epoch in range(epochs):
    loss = 0
    for target_idx, context_idx in training_data:
        # Positive sample
        h = W1[target_idx]
        u = np.dot(h, W2[:, context_idx])
        pred = sigmoid(u)
        error = pred - 1
        loss += -np.log(pred + 1e-7)
        # Gradients
        grad_W2 = error * h
        grad_W1 = error * W2[:, context_idx]
        # Update weights
        W2[:, context_idx] -= learning_rate * grad_W2
        W1[target_idx] -= learning_rate * grad_W1

        # Negative sampling
        negative_samples = random.sample([i for i in range(vocab_size) if i != context_idx], num_negative_samples)
        for neg_idx in negative_samples:
            u_neg = np.dot(h, W2[:, neg_idx])
            pred_neg = sigmoid(u_neg)
            error_neg = pred_neg
            loss += -np.log(1 - pred_neg + 1e-7)
            # Gradients
            grad_W2_neg = error_neg * h
            grad_W1_neg = error_neg * W2[:, neg_idx]
            # Update weights
            W2[:, neg_idx] -= learning_rate * grad_W2_neg
            W1[target_idx] -= learning_rate * grad_W1_neg
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

Epoch 100, Loss: 41.6399
Epoch 200, Loss: 43.7115
Epoch 300, Loss: 46.7961
Epoch 400, Loss: 40.2588
Epoch 500, Loss: 41.3046
Epoch 600, Loss: 36.7148
Epoch 700, Loss: 42.1316
Epoch 800, Loss: 45.5240
Epoch 900, Loss: 40.6802
Epoch 1000, Loss: 43.0892


In [18]:
# Retrieve word embeddings
word_embeddings = W1

# Example: Find similar words
def find_similar(word, top_n=3):
    if word not in word_to_idx:
        print(f"'{word}' not in vocabulary.")
        return
    idx = word_to_idx[word]
    vec = word_embeddings[idx]
    similarities = []
    for i in range(vocab_size):
        if i == idx:
            continue
        sim = np.dot(vec, word_embeddings[i]) / (np.linalg.norm(vec) * np.linalg.norm(word_embeddings[i]))
        similarities.append((idx_to_word[i], sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    for word, sim in similarities[:top_n]:
        print(f"{word}: {sim:.4f}")

# Test the model
print("\nWords similar to 'fox':")
find_similar('fox')


Words similar to 'fox':
jumps: 0.4859
dog: 0.0744
lazy: 0.0424


#### With `Gensim`

In [1]:
import gensim
from gensim.models import Word2Vec

# Sample corpus
sentences = [
    ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"],
    ["i", "love", "natural", "language", "processing"],
    ["word2vec", "is", "a", "technique", "for", "natural", "language", "processing"],
    ["the", "dog", "is", "lazy", "but", "the", "brown", "fox", "is", "quick"]
]


In [2]:
# Initialize and train the model
model = Word2Vec(
    sentences,
    vector_size=100,  # Dimensionality of the word vectors
    window=5,         # Maximum distance between the current and predicted word
    min_count=1,      # Ignores all words with total frequency lower than this
    workers=4,        # Use these many worker threads to train the model
    sg=1              # 1 for Skip-gram; 0 for CBOW
)

In [3]:
# Find most similar words
similar_words = model.wv.most_similar("fox", topn=3)
print(similar_words)

# Compute similarity between two words
similarity = model.wv.similarity("dog", "fox")
print(f"Similarity between 'dog' and 'fox': {similarity:.4f}")

[('over', 0.16699621081352234), ('brown', 0.1388736069202423), ('quick', 0.13150502741336823)]
Similarity between 'dog' and 'fox': -0.1052


In [9]:
sample = """
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.


The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.


When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday our story
starts, there was nothing about the cloudy sky outside to suggest that
strange and mysterious things would soon be happening all over the
country. Mr. Dursley hummed as he picked out his most boring tie for
work, and Mrs. Dursley gossiped away happily as she wrestled a screaming
Dudley into his high chair.
"""

sentences = [
    gensim.utils.simple_preprocess(sentence)
    for sentence in sample.split("\n\n")
]

In [10]:
model = Word2Vec(
    sentences,
    vector_size=100,  # Dimensionality of the word vectors
    window=5,         # Maximum distance between the current and predicted word
    min_count=1,      # Ignores all words with total frequency lower than this
    workers=4,        # Use these many worker threads to train the model
    sg=1              # 1 for Skip-gram; 0 for CBOW
)

In [17]:
# Find most similar words
similar_words = model.wv.most_similar("potter", topn=3)
print(similar_words)

[('dursley', 0.2640940845012665), ('on', 0.2523151636123657), ('starts', 0.24110931158065796)]
